In [5]:
import fastf1
import pandas as pd
import numpy as np
import os

# ================================
# 1. Setup
# ================================
cache_path = "C:/Users/dell/Documents/MECH 534 ML/Project/cache"
os.makedirs(cache_path, exist_ok=True)
fastf1.Cache.enable_cache(cache_path)

# Choose races to include
races = [
    (2023, 'Bahrain', 'R'),
    (2023, 'Monaco', 'R'),
    (2023,'Silverstone','R'),
    (2023,'Monza','R'),
    (2023,'Singapore','R')
]

all_races = []

# ================================
# 2. Loop through races
# ================================
for year, grand_prix, session_type in races:
    print(f"\n🏁 Loading {grand_prix} {year} {session_type}...")
    try:
        session = fastf1.get_session(year, grand_prix, session_type)
        session.load()

        # -------------------------------
        # Base lap data
        # -------------------------------
        laps = session.laps.copy()
        if laps.empty:
            print(f"⚠️ No lap data found for {grand_prix} {year}. Skipping.")
            continue

        laps['LapTime_sec'] = laps['LapTime'].dt.total_seconds()
        for col in ['Sector1Time', 'Sector2Time', 'Sector3Time']:
            laps[col] = laps[col].dt.total_seconds()

        # -------------------------------
        # Weather merge
        # -------------------------------
        weather = session.weather_data.copy()
        weather_cols = [c for c in ['Time', 'AirTemp', 'TrackTemp', 'Humidity',
                                    'Pressure', 'Rainfall', 'WindDirection', 'WindSpeed']
                        if c in weather.columns]
        laps = pd.merge_asof(
            laps.sort_values('Time'),
            weather[weather_cols].sort_values('Time'),
            on='Time',
            direction='nearest'
        )

        # -------------------------------
        # Telemetry aggregation (robust + correct merge)
        # -------------------------------
        telemetry_features = []

        # Build driver lookup (maps numeric ID → abbreviation)
        driver_map = session.results[['Abbreviation']].reset_index()
        driver_map.rename(columns={'index': 'DriverNumber'}, inplace=True)
        driver_map['DriverNumber'] = driver_map['DriverNumber'].astype(str)

        for drv in session.drivers:
            try:
                drv_str = str(drv)
                drv_name = driver_map.loc[driver_map['DriverNumber'] == drv_str, 'Abbreviation']
                if drv_name.empty:
                    continue
                drv_name = drv_name.values[0]  # e.g., "VER"

                drv_laps = laps.pick_driver(drv_name)
                car_data = session.car_data[drv]

                if car_data is None or car_data.empty:
                    print(f"⚠️ No telemetry available for {drv_name}.")
                    continue

                for _, lap in drv_laps.iterlaps():
                    lap_number = lap['LapNumber']
                    lap_start = lap['LapStartTime']
                    lap_end = lap['Time']

                    mask = (car_data['Time'] >= lap_start) & (car_data['Time'] <= lap_end)
                    lap_tel = car_data.loc[mask]

                    if lap_tel.empty:
                        continue

                    lap_stats = {
                        'Driver': drv_name,
                        'LapNumber': lap_number,
                        'Speed_mean': lap_tel['Speed'].mean(),
                        'Speed_max': lap_tel['Speed'].max(),
                        'Throttle_mean': lap_tel['Throttle'].mean(),
                        'Brake_mean': lap_tel['Brake'].mean(),
                        'DRS_activations': lap_tel['DRS'].sum(),
                        'Gear_mean': lap_tel['nGear'].mean()
                    }
                    telemetry_features.append(lap_stats)

                print(f"✅ {drv_name}: telemetry processed ({len(drv_laps)} laps).")

            except Exception as e:
                print(f"⚠️ {drv} telemetry error: {e}")
                continue

        # ✅ Merge telemetry
        if telemetry_features:
            telemetry_df = pd.DataFrame(telemetry_features)
            laps['LapNumber'] = laps['LapNumber'].astype(int)
            telemetry_df['LapNumber'] = telemetry_df['LapNumber'].astype(int)
            laps = pd.merge(laps, telemetry_df, on=['Driver', 'LapNumber'], how='left')
            print(f"✅ Telemetry merged for {len(telemetry_df['Driver'].unique())} drivers.")
        else:
            print("⚠️ No telemetry data for this race; adding NaN columns.")
            for col in ['Speed_mean', 'Speed_max', 'Throttle_mean', 'Brake_mean', 'DRS_activations', 'Gear_mean']:
                laps[col] = np.nan

        # -------------------------------
        # Engineered features
        # -------------------------------
        laps['FuelEffectProxy'] = 1 - (laps['LapNumber'] / laps['LapNumber'].max())
        laps['LapProgress'] = laps['LapNumber'] / laps['LapNumber'].max()
        laps['CumulativeTireWear'] = laps['TyreLife'] ** 2
        laps['AvgSectorTime'] = laps[['Sector1Time', 'Sector2Time', 'Sector3Time']].mean(axis=1)

        # Metadata
        laps['Race'] = grand_prix
        laps['Year'] = year
        laps['SessionType'] = session_type

        # -------------------------------
        # Select final columns
        # -------------------------------
        final_cols = [
            'Driver', 'Team', 'Year', 'Race', 'SessionType', 'LapNumber', 'Stint',
            'Compound', 'TyreLife', 'CumulativeTireWear', 'LapTime_sec', 'TrackStatus',
            'Position', 'IsPersonalBest', 'FuelEffectProxy', 'LapProgress', 'AvgSectorTime',
            'Speed_mean', 'Speed_max', 'Throttle_mean', 'Brake_mean', 'DRS_activations',
            'Gear_mean', 'AirTemp', 'TrackTemp', 'Humidity', 'Pressure', 'Rainfall',
            'WindDirection', 'WindSpeed'
        ]

        laps_final = laps[final_cols].copy()
        print(f"✅ Finished processing {grand_prix} {year}: {len(laps_final)} laps extracted.")

        if not laps_final.empty:
            all_races.append(laps_final)

    except Exception as e:
        print(f"❌ Failed to process {grand_prix} {year}: {e}")
        continue

# ================================
# 3. Combine and save
# ================================
if all_races:
    combined_df = pd.concat(all_races, ignore_index=True)
    output_path = "C:/Users/dell/Documents/MECH 534 ML/Project/F1_five_races_full_features.csv"
    combined_df.to_csv(output_path, index=False)

    print(f"\n✅ Combined dataset saved to: {output_path}")
    print(f"Total laps: {combined_df.shape[0]} | Features: {combined_df.shape[1]}")
else:
    print("❌ No valid race data to combine!")



core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data



🏁 Loading Bahrain 2023 R...


req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pi

✅ VER: telemetry processed (57 laps).
✅ PER: telemetry processed (57 laps).
✅ ALO: telemetry processed (57 laps).


c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


...

✅ BOT: telemetry processed (52 laps).


c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


✅ OCO: telemetry processed (43 laps).
✅ TSU: telemetry processed (1 laps).
✅ STR: telemetry processed (0 laps).
✅ Telemetry merged for 19 drivers.
✅ Finished processing Singapore 2023: 1088 laps extracted.


c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"



✅ Combined dataset saved to: C:/Users/dell/Documents/MECH 534 ML/Project/F1_five_races_full_features.csv
Total laps: 5588 | Features: 30
